In [ ]:

from pyspark import SparkConf
from pyspark.streaming import StreamingContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import window, col, avg, concat, lit
from pyspark.sql.types import StructType, StructField, LongType, StringType, DoubleType, IntegerType
from time import sleep
import pandas as pd
from google.cloud import pubsub_v1

In [ ]:
sparkConf = SparkConf()
sparkConf.setMaster("spark://spark-master:7077")
sparkConf.setAppName("CrimeStreamProcessing")
sparkConf.set("spark.driver.memory", "2g")
sparkConf.set("spark.executor.cores", "1")
sparkConf.set("spark.driver.cores", "1")

# create the spark session, which is the entry point to Spark SQL engine.
spark = SparkSession.builder.config(conf=sparkConf).getOrCreate()

project_number = 868393134342
location = "us-central1-a"
subscription_id = "crimes-subscription"

sdf = (
    spark.readStream.format("pubsublite")
    .option(
        "pubsublite.subscription",
        f"projects/{project_number}/locations/{location}/subscriptions/{subscription_id}",
    )
    .load()
)

sdf = sdf.withColumn("data", sdf.data.cast(StringType()))

print(sdf.head())

query = (
    sdf.writeStream.format("console")
    .outputMode("append")
    .trigger(processingTime="1 second")
    .start()
)

query.awaitTermination(60)
query.stop()


# ssc = StreamingContext(sparkConf, 28) # https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.streaming.StreamingContext.html





# with pubsub_v1.SubscriberClient() as subscriber:
#     subscriber.create_subscription(
#         name=subscription_name, topic=topic_name)
#     future = subscriber.subscribe(subscription_name, callback)

In [ ]:

# dataSchema = StructType(
#     [StructField("uname", StringType(), True),
#      StructField("tname", StringType(), True),
#      StructField("score", IntegerType(), True),
#      StructField("timestamp_in_ms", LongType(), True),
#      StructField("readable_time", StringType(), True)
#      ])

# # Read from a source 
# sdf = spark.readStream.schema(dataSchema).option("maxFilesPerTrigger", 1) \
#     .csv("/home/jovyan/data/gamescore")
    
    
# # create the event time column 
# withEventTimedf = sdf.selectExpr(
#     "*",
#     "cast(timestamp_in_ms/1000.0 as timestamp) as event_time")

# withEventTimedf.printSchema()

# avgscoredf = withEventTimedf \
#     .groupBy(window(col("event_time"), "10 seconds"), "uname", "tname") \
#     .agg(avg("score").alias("value"))

# resultdf = avgscoredf.select(concat(col("uname"), lit(" "), col("tname")).alias("key"), col("value"))

# query = resultdf \
#     .writeStream \
#     .queryName("avg_score_window") \
#     .format("memory") \
#     .outputMode("complete") \
#     .start()

# try:
#     for x in range(100):
#         spark.sql("SELECT * FROM avg_score_window").show()
#         sleep(10)
# except KeyboardInterrupt:
#     query.stop()
#     # Stop the spark context
#     spark.stop()
#     print("Stoped the streaming query and the spark context")